# Prueba de varios modelos HMM tras la aplicación del mejor estimador

In [27]:
#Lectura de la base de datos
import pandas as pd

#Preprocesamiento de los datos
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer

#Evaluación de la actuación del modelo
from sklearn import metrics

#Modelos HMM
import seqlearn.hmm
from seqlearn.hmm import MultinomialHMM
import seqlearn.perceptron
from seqlearn.perceptron import StructuredPerceptron
from pomegranate import *

#Carga y almacenamiento de modelos
import json
import pickle

In [28]:
#Lectura del fichero de datos CSV. Separación por ; e ignorando el espacio inicial
data = pd.read_csv('database_speech.csv', sep = ';', skipinitialspace = True)

#Eliminacion de fases
#data = data.drop(data[data['Phase']==8].index)

#Muestra de datos cargados (10)
#data.head(10)

#Inicialización de los métodos de sobremuestreo
#ROS
ros = RandomOverSampler()
#SMOTE
smote = SMOTE()

#Inicialización de los vectores de extracción de características. Solo se usa TfidfVectorizer porque el mejor 
#estimador utilizaba este vector de características. No puede usar CountVectorizer si fue entrenado con otro vector.
#TfidfVectorizer. Frecuencia mínima de repetición de 3, sin ponderación inversa de la frecuencia (los que menos
#se repiten más peso tienen) ni suavizado (el mínimo de apariciones es 1 aunque la palabra no esté en la frase),
#palabras de parada definidas y ajuste a las transcripciones
vectTf = TfidfVectorizer(min_df=3, use_idf = False, smooth_idf = False,stop_words = ['a', 'aca','acá', 'ahi','ahí', 'ajena', 'ajeno', 'ajenas', 'ajenos', 'al', 'algo', 'algun','algún', 'alguna', 'alguno', 'algunas', 'algunos', 'alla','allá','allí', 'alli', 'ambos', 'ante', 'antes', 'aquel', 'aquella', 'aquello', 'aquellas', 'aquellos', 'aqui','aquí', 'arriba', 'asi','así', 'atras', 'aun', 'aunque', 'bajo', 'bastante', 'bien', 'cabe', 'cada', 'casi', 'cierto', 'cierta', 'ciertos', 'ciertas', 'como', 'con', 'conmigo', 'conseguimos', 'conseguir', 'consigo', 'consigue', 'consiguen', 'consigues', 'contigo', 'contra', 'cual', 'cuales', 'cualquier', 'cualquiera', 'cualquieras', 'cuan', 'cuando', 'cuanto', 'cuanta', 'cuantos', 'cuantas', 'de', 'dejar', 'del', 'demas','demás', 'demasiada', 'demasiado', 'demasiadas', 'demasiados', 'dentro', 'desde', 'donde', 'dos', 'el', 'él','ella', 'ello', 'ellas', 'ellos', 'empleais','empleáis', 'emplean', 'emplear', 'empleas', 'empleo', 'en', 'encima', 'entonces', 'entre', 'era', 'eras', 'eramos', 'eran', 'eres', 'es', 'esa', 'ese', 'eso', 'esas', 'eses', 'esos', 'esta', 'estas', 'estaba', 'estado', 'estais','estáis', 'estamos', 'estan', 'estar', 'este','están', 'esto', 'estos', 'estes', 'estoy', 'etc', 'fin', 'fue', 'fueron', 'fui', 'fuimos', 'gueno', 'ha', 'hace', 'haces', 'haceis','hacéis', 'hacemos', 'hacen', 'hacer', 'hacia', 'hago', 'hasta', 'incluso', 'intenta', 'intentas', 'intentais', 'intentáis', 'intentamos', 'intentan', 'intentar', 'intento', 'ir', 'jamas','jamás', 'junto', 'juntos', 'la', 'lo', 'las', 'los', 'largo', 'mas','más', 'me', 'menos', 'mi', 'mis', 'mia', 'mias','mías', 'mía', 'mientras', 'mio', 'mios','míos', 'mío', 'misma', 'mismo', 'mismas', 'mismos', 'modo', 'mucha', 'muchas', 'muchisima', 'muchisimo', 'muchisimas', 'muchisimos','muchísima', 'muchísimo', 'muchísimas', 'muchísimos', 'mucho', 'muchos', 'muy', 'nada', 'ni', 'ningun', 'ningún', 'ninguna', 'ninguno', 'ningunas', 'ningunos', 'no', 'nos', 'nosotras', 'nosotros', 'nuestra', 'nuestro', 'nuestras', 'nuestros', 'nunca', 'os', 'otra', 'otro', 'otras', 'otros', 'para', 'parecer', 'pero', 'poca', 'poco', 'pocas', 'pocos', 'podeis','podéis', 'podemos', 'poder', 'podria', 'podrias', 'podriais', 'podriamos', 'podrian','podría', 'podrías', 'podríais', 'podríamos','podrían', 'por', 'por que','por qué', 'porque', 'primero', 'puede', 'pueden', 'puedo', 'pues', 'que', 'qué', 'querer', 'quien', 'quién', 'quienes', 'quienesquiera', 'quienquiera', 'quiza', 'quizá', 'quizás', 'quizas', 'sabe', 'sabes', 'saben', 'sabeis', 'sabéis', 'sabemos', 'saber', 'se', 'segun', 'según', 'ser','sí', 'si', 'siempre', 'siendo', 'sin', 'sino', 'so', 'sobre', 'sois', 'solamente', 'solo','sólo', 'somos', 'soy', 'sr', 'sra', 'sres', 'sta', 'su', 'sus', 'suya', 'suyo', 'suyas', 'suyos', 'tal', 'tales', 'tambien','también', 'tampoco', 'tan', 'tanta', 'tanto', 'tantas', 'tantos', 'te', 'teneis','tenéis', 'tenemos', 'tener', 'tengo', 'ti', 'tiempo', 'tiene', 'tienen', 'toda', 'todo', 'todas', 'todos', 'tomar', 'trabaja', 'trabajo', 'trabajais', 'trabajáis', 'trabajamos', 'trabajan', 'trabajar', 'trabajas', 'tras', 'tu','tú', 'tus', 'tuya', 'tuyo', 'tuyas', 'tuyos', 'ultimo', 'último', 'un', 'una', 'unos', 'unas', 'uno', 'usa', 'usas', 'usais','usáis', 'usamos', 'usan', 'usar', 'uso', 'usted', 'ustedes', 'va', 'van', 'vais', 'valor', 'vamos', 'varias', 'varios', 'vaya', 'verdadera', 'vosotras', 'vosotros', 'voy', 'vuestra', 'vuestro', 'vuestras', 'vuestros', 'y', 'ya', 'yo']).fit(data['Transcription']) 

#Obtención de las matrices de características
X_vectorizedTf = vectTf.transform(data['Transcription'])

#Balanceo de datos
X_TfSM, y_TfSM = ros.fit_resample(X_vectorizedTf,data['Phase'])
X_TfRO, y_TfRO = smote.fit_resample(X_vectorizedTf,data['Phase'])

#División en conjunto de entrenamiento y conjunto de evaluación
X_trainTfRO, X_testTfRO, y_trainTfRO, y_testTfRO = train_test_split(X_TfRO, y_TfRO)
X_trainTfSM, X_testTfSM, y_trainTfSM, y_testTfSM = train_test_split(X_TfSM, y_TfSM)

In [33]:
#Carga del mejor estimador almacenado previamente
filename = 'modelSVMDEFSVCTfSM.model'
sModel = pickle.load(open(filename, 'rb'))

#Calculo de las probabilidades de cada fase por el mejor estimador
predictions = sModel.predict_proba(X_trainTfRO)

#Inicialización de los modelos de HMM
modelHMM1 = MultinomialHMM(alpha = 300,decode = 'bestfirst')
modelHMM2 = MultinomialHMM(alpha = 10,decode = 'bestfirst')
modelHMM3 = MultinomialHMM(alpha = 5,decode = 'bestfirst')
modelHMM4 = MultinomialHMM(alpha = 1,decode = 'bestfirst')
modelHMM5 = MultinomialHMM(alpha = 0.5,decode = 'bestfirst')
modelHMM6 = MultinomialHMM(alpha = 0.1,decode = 'bestfirst')
modelHMM7 = MultinomialHMM(alpha = 0.05,decode = 'bestfirst')
modelHMM8 = MultinomialHMM(alpha = 0.01,decode = 'bestfirst')
modelHMM9 = MultinomialHMM(alpha = 0.005,decode = 'bestfirst')
modelHMM10 = MultinomialHMM(alpha = 0.001,decode = 'bestfirst')
modelHMM11 = MultinomialHMM(alpha = 0.0005,decode = 'bestfirst')
modelHMM12 = MultinomialHMM(alpha = 0.0001,decode = 'bestfirst')
modelHMM13 = MultinomialHMM(alpha = 15,decode = 'viterbi')
modelHMM14 = MultinomialHMM(alpha = 10,decode = 'viterbi')
modelHMM15 = MultinomialHMM(alpha = 5,decode = 'viterbi')
modelHMM16 = MultinomialHMM(alpha = 1,decode = 'viterbi')
modelHMM17 = MultinomialHMM(alpha = 0.5,decode = 'viterbi')
modelHMM18 = MultinomialHMM(alpha = 0.1,decode = 'viterbi')
modelHMM19 = MultinomialHMM(alpha = 0.05,decode = 'viterbi')
modelHMM20 = MultinomialHMM(alpha = 0.01,decode = 'viterbi')
modelHMM21 = MultinomialHMM(alpha = 0.005,decode = 'viterbi')
modelHMM22 = MultinomialHMM(alpha = 0.001,decode = 'viterbi')
modelHMM23 = MultinomialHMM(alpha = 0.0005,decode = 'viterbi')
modelHMM24 = MultinomialHMM(alpha = 0.0001,decode = 'viterbi')

#Ajuste del modelo a las probabilidades y la fase
modelHMM1.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM2.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM3.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM4.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM5.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM6.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM7.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM8.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM9.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM10.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM11.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM12.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM13.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM14.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM15.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM16.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM17.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM18.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM19.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM20.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM21.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM22.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM23.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM24.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])

#Predicciones con el modelo de HMM
pred = sModel.predict_proba(X_testTfRO)

y_pred1 = modelHMM1.predict(pred,[len(y_testTfRO)])
y_pred2 = modelHMM2.predict(pred,[len(y_testTfRO)])
y_pred3 = modelHMM3.predict(pred,[len(y_testTfRO)])
y_pred4 = modelHMM4.predict(pred,[len(y_testTfRO)])
y_pred5 = modelHMM5.predict(pred,[len(y_testTfRO)])
y_pred6 = modelHMM6.predict(pred,[len(y_testTfRO)])
y_pred7 = modelHMM7.predict(pred,[len(y_testTfRO)])
y_pred8 = modelHMM8.predict(pred,[len(y_testTfRO)])
y_pred9 = modelHMM9.predict(pred,[len(y_testTfRO)])
y_pred10 = modelHMM10.predict(pred,[len(y_testTfRO)])
y_pred11 = modelHMM11.predict(pred,[len(y_testTfRO)])
y_pred12 = modelHMM12.predict(pred,[len(y_testTfRO)])
y_pred13 = modelHMM13.predict(pred,[len(y_testTfRO)])
y_pred14 = modelHMM14.predict(pred,[len(y_testTfRO)])
y_pred15 = modelHMM15.predict(pred,[len(y_testTfRO)])
y_pred16 = modelHMM16.predict(pred,[len(y_testTfRO)])
y_pred17 = modelHMM17.predict(pred,[len(y_testTfRO)])
y_pred18 = modelHMM18.predict(pred,[len(y_testTfRO)])
y_pred19 = modelHMM19.predict(pred,[len(y_testTfRO)])
y_pred20 = modelHMM20.predict(pred,[len(y_testTfRO)])
y_pred21 = modelHMM21.predict(pred,[len(y_testTfRO)])
y_pred22 = modelHMM22.predict(pred,[len(y_testTfRO)])
y_pred23 = modelHMM23.predict(pred,[len(y_testTfRO)])
y_pred24 = modelHMM24.predict(pred,[len(y_testTfRO)])

#Evaluación de los resultados del modelo
print(metrics.f1_score(y_testTfRO,y_pred1,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred2,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred3,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred4,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred5,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred6,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred7,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred8,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred9,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred10,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred11,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred12,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred13,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred14,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred15,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred16,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred17,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred18,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred19,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred20,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred21,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred22,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred23,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred24,average = 'weighted'))

#Descomentar si se desea salvar el mejor modelo 
#filename = 'modelHMM.model'
#pickle.dump(modelHMM1, open(filename, 'wb'))

0.9262060873626494
0.9268149498229367
0.9309702495226245
0.9187228984285273
0.9148449644124522
0.910955506741329
0.910955506741329
0.9071310793318518
0.9071310793318518
0.9034657602526469
0.9034657602526469
0.9034657602526469
0.9267756140590672
0.9267756140590672
0.9228513758561062
0.9189225846359158
0.9189225846359158
0.9189225846359158
0.9189225846359158
0.9150030438491766
0.9150030438491766
0.9112467178750303
0.9112467178750303
0.9112467178750303


In [34]:
#Carga del mejor estimador almacenado previamente
filename = 'modelSVMDEFSVCTfSM.model'
sModel = pickle.load(open(filename, 'rb'))

#Calculo de las probabilidades de cada fase por el mejor estimador
predictions = sModel.predict_proba(X_trainTfRO)

#Inicialización modelos HMM
modelHMM1 = StructuredPerceptron(lr_exponent = 15,decode = 'bestfirst', max_iter = 1000)
modelHMM2 = StructuredPerceptron(lr_exponent = 10,decode = 'bestfirst', max_iter = 1000)
modelHMM3 = StructuredPerceptron(lr_exponent = 5,decode = 'bestfirst', max_iter = 1000)
modelHMM4 = StructuredPerceptron(lr_exponent = 1,decode = 'bestfirst', max_iter = 1000)
modelHMM5 = StructuredPerceptron(lr_exponent = 0.5,decode = 'bestfirst', max_iter = 1000)
modelHMM6 = StructuredPerceptron(lr_exponent = 0.1,decode = 'bestfirst', max_iter = 1000)
modelHMM7 = StructuredPerceptron(lr_exponent = 0.05,decode = 'bestfirst', max_iter = 1000)
modelHMM8 = StructuredPerceptron(lr_exponent = 0.01,decode = 'bestfirst', max_iter = 1000)
modelHMM9 = StructuredPerceptron(lr_exponent = 0.005,decode = 'bestfirst', max_iter = 1000)
modelHMM10 = StructuredPerceptron(lr_exponent = 0.001,decode = 'bestfirst', max_iter = 1000)
modelHMM11 = StructuredPerceptron(lr_exponent = 0.0005,decode = 'bestfirst', max_iter = 1000)
modelHMM12 = StructuredPerceptron(lr_exponent = 0.0001,decode = 'bestfirst', max_iter = 1000)
modelHMM13 = StructuredPerceptron(lr_exponent = 15,decode = 'viterbi', max_iter = 1000)
modelHMM14 = StructuredPerceptron(lr_exponent = 10,decode = 'viterbi', max_iter = 1000)
modelHMM15 = StructuredPerceptron(lr_exponent = 5,decode = 'viterbi', max_iter = 1000)
modelHMM16 = StructuredPerceptron(lr_exponent = 1,decode = 'viterbi', max_iter = 1000)
modelHMM17 = StructuredPerceptron(lr_exponent = 0.5,decode = 'viterbi', max_iter = 1000)
modelHMM18 = StructuredPerceptron(lr_exponent = 0.1,decode = 'viterbi', max_iter = 1000)
modelHMM19 = StructuredPerceptron(lr_exponent = 0.05,decode = 'viterbi', max_iter = 1000)
modelHMM20 = StructuredPerceptron(lr_exponent = 0.01,decode = 'viterbi', max_iter = 1000)
modelHMM21 = StructuredPerceptron(lr_exponent = 0.005,decode = 'viterbi', max_iter = 1000)
modelHMM22 = StructuredPerceptron(lr_exponent = 0.001,decode = 'viterbi', max_iter = 1000)
modelHMM23 = StructuredPerceptron(lr_exponent = 0.0005,decode = 'viterbi', max_iter = 1000)
modelHMM24 = StructuredPerceptron(lr_exponent = 0.0001,decode = 'viterbi', max_iter = 1000)

#Entrenamiento modelos HMM
modelHMM1.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM2.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM3.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM4.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM5.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM6.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM7.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM8.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM9.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM10.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM11.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM12.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM13.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM14.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM15.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM16.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM17.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM18.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM19.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM20.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM21.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM22.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM23.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])
modelHMM24.fit(predictions,y_trainTfRO,[len(y_trainTfRO)])

#Predicción con el modelo HMM
pred = sModel.predict_proba(X_testTfRO)

y_pred1 = modelHMM1.predict(pred,[len(y_testTfRO)])
y_pred2 = modelHMM2.predict(pred,[len(y_testTfRO)])
y_pred3 = modelHMM3.predict(pred,[len(y_testTfRO)])
y_pred4 = modelHMM4.predict(pred,[len(y_testTfRO)])
y_pred5 = modelHMM5.predict(pred,[len(y_testTfRO)])
y_pred6 = modelHMM6.predict(pred,[len(y_testTfRO)])
y_pred7 = modelHMM7.predict(pred,[len(y_testTfRO)])
y_pred8 = modelHMM8.predict(pred,[len(y_testTfRO)])
y_pred9 = modelHMM9.predict(pred,[len(y_testTfRO)])
y_pred10 = modelHMM10.predict(pred,[len(y_testTfRO)])
y_pred11 = modelHMM11.predict(pred,[len(y_testTfRO)])
y_pred12 = modelHMM12.predict(pred,[len(y_testTfRO)])
y_pred13 = modelHMM13.predict(pred,[len(y_testTfRO)])
y_pred14 = modelHMM14.predict(pred,[len(y_testTfRO)])
y_pred15 = modelHMM15.predict(pred,[len(y_testTfRO)])
y_pred16 = modelHMM16.predict(pred,[len(y_testTfRO)])
y_pred17 = modelHMM17.predict(pred,[len(y_testTfRO)])
y_pred18 = modelHMM18.predict(pred,[len(y_testTfRO)])
y_pred19 = modelHMM19.predict(pred,[len(y_testTfRO)])
y_pred20 = modelHMM20.predict(pred,[len(y_testTfRO)])
y_pred21 = modelHMM21.predict(pred,[len(y_testTfRO)])
y_pred22 = modelHMM22.predict(pred,[len(y_testTfRO)])
y_pred23 = modelHMM23.predict(pred,[len(y_testTfRO)])
y_pred24 = modelHMM24.predict(pred,[len(y_testTfRO)])

#Evaluación de los resultados del modelo
print(metrics.f1_score(y_testTfRO,y_pred1,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred2,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred3,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred4,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred5,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred6,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred7,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred8,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred9,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred10,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred11,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred12,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred13,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred14,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred15,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred16,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred17,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred18,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred19,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred20,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred21,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred22,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred23,average = 'weighted'))
print(metrics.f1_score(y_testTfRO,y_pred24,average = 'weighted'))

#Descomentar si se desea salvar el mejor modelo 
#filename = 'modelHMM.model'
#pickle.dump(modelHMM1, open(filename, 'wb'))

0.7521401397784644
0.7512303493247638
0.7442046104166014
0.879804751140046
0.8617331080565759
0.9145442338329278
0.9145442338329278
0.9104623831587352
0.9104623831587352
0.9063751139880257
0.9104623831587352
0.9104623831587352
0.7473346269317126
0.7473346269317126
0.7463352798577024
0.8765174638295159
0.8528045293518606
0.914711564898016
0.9148838455077192
0.9111642089116356
0.9069699337980036
0.9111642089116356
0.9069699337980036
0.9068144319974428


In [35]:
#Carga del mejor estimador almacenado previamente
filename = 'modelSVMDEFSVCTfSM.model'
sModel = pickle.load(open(filename, 'rb'))

#Calculo de las probabilidades de cada fase por el mejor estimador
predictions = sModel.predict_proba(X_trainTfRO)

#Creación del modelo. Distribución de Dirichlet, 8 estados y algoritmo de aprendizaje supervisado ('labeled')
modelHMM = HiddenMarkovModel.from_samples(DirichletDistribution, n_components = 8, X = [predictions], labels = [y_trainTfRO-1], algorithm = 'labeled',name = 'HMM', n_jobs = -1)

#Predicción con el HMM
pred = sModel.predict_proba(X_testTfRO)
y_testTfRO, y_predTfRO = y_testTfRO, modelHMM.predict(pred,algorithm = 'labeled')

#Elimina el estado de inicio generado por el modelo por defecto
y_predTfRO.pop(0)

#Obtenemos la métrica
print(metrics.f1_score(y_testTfRO - 1,y_predTfRO,average = 'weighted'))

#Almacenamos el modelo generado
#jeson = modelHMM.to_json()
#with open('HMM.json','w') as file:
#    json.dump(jeson,file)

KeyError: 1

Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize'
KeyError: 1


KeyError: 1

0.9229873104182066


Exception ignored in: 'pomegranate.hmm.HiddenMarkovModel._labeled_summarize'
KeyError: 1
